# Import the required libraries to extract data from twitter and for data preprocessing.

In [1]:
import re
import csv
import tweepy
from textblob import TextBlob
from tweepy import OAuthHandler
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Step 1: Getting Twitter API keys
In order to access Twitter Streaming API, we need to get 4 pieces of information from Twitter: API key, API secret, Access token and Access token secret. Follow the steps below to get all 4 elements:

1) Create a twitter account if you do not already have one.
2) Go to https://apps.twitter.com/ and log in with your twitter credentials.
3) Click "Create New App"
4) Fill out the form, agree to the terms, and click "Create your Twitter application"
5) In the next page, click on "API keys" tab, and copy your "API key" and "API secret".
6) Scroll down and click "Create my access token", and copy your "Access token" and "Access token secret".

# Step 2: Connecting to Twitter Streaming API and downloading data

In [2]:
consumer_key = 'L5a3avLNGPXmKw3PD313KKN7t'
consumer_secret = 'oPP1FKnYU2wVVluyNBH0EOLqw0TNgVxS82wubNU7gNDtRCMjT4'
access_token = '968883773495443458-FcbinMeZCT9D4x3LjmumuvzkfXiNJLP'
access_secret = 'UfSHf6c88NPBpOaXpVqL8X6TJSQoRpNibTXKXJahD2LDd'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Step 3 : Extact the data and write it to a CSV file

In [3]:
# Open/create a file to append data to
csvFile = open('13ReasonsWhy_1.csv', 'a')

#Use csv writer
csvWriter = csv.writer(csvFile)

for tweet in tweepy.Cursor(api.search,
                           q = "13ReasonsWhy",
                           since = "2018-04-23",
                           until = "2018-04-24",
                           lang = "en").items():

    # Write a row to the CSV file. I use encode UTF-8
    #csvWriter.writerow([tweet.text.encode('utf-8')])
    #csvWriter.writerow([tweet.text.encode('utf-8')])
    print (tweet.text)
    print(tweet.get)
csvFile.close()

In [4]:
twitter = pd.read_csv("C://Users//Aswathi//13ReasonsWhy.csv",names = {"Tweet"})

# Step 4: Data Cleaning by removing symbols, @ and hashtags

In [6]:
twitter.head()

,Tweet
0,b'@johnkrasinski I saw @quietplacemovie on mon...
1,b'RT @quietplacemovie: Silence is golden. Expe...
2,b'If you wanna know how they felt in @quietpla...
3,b'RT @quietplacemovie: \xe2\x80\x9cMy job is t...
4,b'@capegirl8 @quietplacemovie Always down for ...


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aswathi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
l = []
for i in range(0,len(twitter)):
    data = re.sub('[^a-zA-Z]',' ',twitter.loc[i,"Tweet"])
    data = data[2:]
    data = data.lower()
    data = word_tokenize(data)
    lem = WordNetLemmatizer()
    data = [lem.lemmatize(word) for word in data if not word in set(stopwords.words('english'))]
    data = ' '.join(data)
    l.append(data)

In [9]:
twitter['clean_Tweet'] = pd.DataFrame(l)

In [10]:
twitter.head()

,Tweet,clean_Tweet
0,b'@johnkrasinski I saw @quietplacemovie on mon...,johnkrasinski saw quietplacemovie monday tryin...
1,b'RT @quietplacemovie: Silence is golden. Expe...,rt quietplacemovie silence golden experience a...
2,b'If you wanna know how they felt in @quietpla...,wan na know felt quietplacemovie order l popco...
3,b'RT @quietplacemovie: \xe2\x80\x9cMy job is t...,rt quietplacemovie xe x x cmy job tell story f...
4,b'@capegirl8 @quietplacemovie Always down for ...,capegirl quietplacemovie always movie date xf ...


In [11]:
df_twitter = pd.DataFrame(twitter['clean_Tweet'])

In [12]:
df_twitter = df_twitter.drop_duplicates().reset_index()

In [13]:
df_twitter = pd.DataFrame(twitter['clean_Tweet'])

In [14]:
df_twitter.head()

,clean_Tweet
0,johnkrasinski saw quietplacemovie monday tryin...
1,rt quietplacemovie silence golden experience a...
2,wan na know felt quietplacemovie order l popco...
3,rt quietplacemovie xe x x cmy job tell story f...
4,capegirl quietplacemovie always movie date xf ...


# Step 5 : Calculate the polarity of the tweets using TextBlob package

In [15]:
for i in range(0,len(df_twitter)):
    blob = TextBlob(df_twitter.loc[i,"clean_Tweet"])
    df_twitter.loc[i,'polarity'] = blob.sentiment.polarity

In [16]:
df_twitter.head()

,clean_Tweet,polarity
0,johnkrasinski saw quietplacemovie monday tryin...,0.000000
1,rt quietplacemovie silence golden experience a...,0.300000
2,wan na know felt quietplacemovie order l popco...,0.066667
3,rt quietplacemovie xe x x cmy job tell story f...,0.285714
4,capegirl quietplacemovie always movie date xf ...,0.000000


# Step 6 : Convert the polarity into sentiments. 

In [17]:
for i in range(0,len(df_twitter)):
    if(df_twitter.loc[i,"polarity"] > 0) :
        df_twitter.loc[i,'sentiment'] = 1
    else:
        if(df_twitter.loc[i,"polarity"] < 0):
            df_twitter.loc[i,'sentiment'] = -1
        else:
            df_twitter.loc[i,'sentiment'] = 0

In [18]:
df_twitter.head()

,clean_Tweet,polarity,sentiment
0,johnkrasinski saw quietplacemovie monday tryin...,0.000000,0.0
1,rt quietplacemovie silence golden experience a...,0.300000,1.0
2,wan na know felt quietplacemovie order l popco...,0.066667,1.0
3,rt quietplacemovie xe x x cmy job tell story f...,0.285714,1.0
4,capegirl quietplacemovie always movie date xf ...,0.000000,0.0


# Step 7 : Analyzing the results

In [19]:
pos_tweets = [ tweet for index, tweet in enumerate(df_twitter['clean_Tweet']) if df_twitter['sentiment'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(df_twitter['clean_Tweet']) if df_twitter['sentiment'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(df_twitter['clean_Tweet']) if df_twitter['sentiment'][index] < 0]

In [20]:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(df_twitter['clean_Tweet'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(df_twitter['clean_Tweet'])))
print("Percentage of negative tweets: {}%".format(len(neg_tweets)*100/len(df_twitter['clean_Tweet'])))

Percentage of positive tweets: 59.336609336609335%
Percentage of neutral tweets: 32.98525798525799%
Percentage of negative tweets: 7.678132678132678%


# Looks like the audience really loved the movie. Most of the tweets are postive, we have very less negative tweets. The dataset looks imbalanced

In [21]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import metrics
import nltk

In [22]:
X = df_twitter['clean_Tweet'].values
y = df_twitter['sentiment'].values

# The classification algorithm will need feature vectors, therefore we will convert our tweets into the Bag of Words (BoW) Model

In [23]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 123)

In [24]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)        
        
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [25]:
X_new_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_new_counts)

# Naive Bayes Classifier 

In [77]:
prediction = pd.DataFrame(y_test,columns = ['y_test'])

In [78]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train_tfidf,y_train)
prediction['Naive_Bayes'] = classifier.predict(X_test_tfidf)

In [85]:
cm = confusion_matrix(y_test,prediction['Naive_Bayes'])
cm

array([[  0,   3,  26],
       [  0,  53,  61],
       [  0,   3, 180]], dtype=int64)

In [86]:
print(classification_report(y_test, prediction['Naive_Bayes']))

             precision    recall  f1-score   support

       -1.0       0.00      0.00      0.00        29
        0.0       0.90      0.46      0.61       114
        1.0       0.67      0.98      0.80       183

avg / total       0.69      0.71      0.66       326



# Postive Examples

In [29]:
tweet_pos = ["AQuietPlace is blowing past expectations and is now looking at an opening weekend around $40 million"]

In [30]:
tweet_count_pos = count_vect.transform(tweet_pos)
tweet_transformed = tfidf_transformer.transform(tweet_count_pos)

In [31]:
classifier.predict(tweet_transformed)[0]

1.0

Even though the below tweet have words like scariest, afraid, scream the classifier is classifying it as positive, because the model has very little negative data to learn from. Therefore the imbalance has caused the data to overfit and thus predict badly on test data

In [32]:
tweet_neg = ["@quietplacemovie was the scariest horror film I’ve ever seen. I was afraid to scream."]

In [33]:
tweet_count_neg = count_vect.transform(tweet_neg)
tweet_transformed = tfidf_transformer.transform(tweet_count_neg)

In [34]:
classifier.predict(tweet_transformed)[0]

1.0

# Decision Tree  

In [35]:
from sklearn import tree

In [46]:
tree_classifier = tree.DecisionTreeClassifier(criterion = "entropy", random_state =100)

In [80]:
tree_classifier.fit(X_train_tfidf,y_train)
prediction['Decision_Tree'] = tree_classifier.predict(X_test_tfidf)
 

In [82]:
cm = confusion_matrix(y_test,prediction['Decision_Tree'])
cm

array([[  7,  14,   8],
       [  4,  89,  21],
       [  3,  40, 140]], dtype=int64)

In [84]:
print(classification_report(y_test,prediction['Decision_Tree']))

             precision    recall  f1-score   support

       -1.0       0.50      0.24      0.33        29
        0.0       0.62      0.78      0.69       114
        1.0       0.83      0.77      0.80       183

avg / total       0.73      0.72      0.72       326



# Random Forest

In [90]:
from sklearn.ensemble import RandomForestClassifier

In [91]:
rf_classifier = RandomForestClassifier(max_depth=2,random_state=100)

In [92]:
rf_classifier.fit(X_train_tfidf,y_train)
prediction['Random_Forest'] = rf_classifier.predict(X_test_tfidf)

In [93]:
cm = confusion_matrix(y_test,prediction['Random_Forest'])
cm

array([[  0,   0,  29],
       [  0,  23,  91],
       [  0,   0, 183]], dtype=int64)

In [94]:
print(classification_report(y_test, prediction['Random_Forest']))

             precision    recall  f1-score   support

       -1.0       0.00      0.00      0.00        29
        0.0       1.00      0.20      0.34       114
        1.0       0.60      1.00      0.75       183

avg / total       0.69      0.63      0.54       326



# Support Vector Machine(SVM)

In [56]:
from sklearn import svm

In [130]:
svm_classifier = svm.SVC(kernel = 'linear',class_weight = 'balanced',random_state=100)

In [131]:
svm_classifier.fit(X_train_tfidf,y_train)
prediction['SVM'] = svm_classifier.predict(X_test_tfidf)

In [132]:
cm = confusion_matrix(y_test,prediction['SVM'])
cm

array([[ 10,  15,   4],
       [  1, 105,   8],
       [  4,  25, 154]], dtype=int64)

In [135]:
print(classification_report(y_test, prediction['SVM']))

             precision    recall  f1-score   support

       -1.0       0.67      0.34      0.45        29
        0.0       0.72      0.92      0.81       114
        1.0       0.93      0.84      0.88       183

avg / total       0.83      0.83      0.82       326



In [142]:
prediction.sample(30)

,y_test,Naive_Bayes,Decision_Tree,SVM,Random_Forest
273,1.0,1.0,1.0,1.0,1.0
272,1.0,1.0,1.0,1.0,1.0
187,-1.0,1.0,0.0,0.0,1.0
50,1.0,1.0,1.0,1.0,1.0
52,1.0,1.0,1.0,0.0,1.0
55,1.0,1.0,1.0,1.0,1.0
85,1.0,1.0,1.0,1.0,1.0
172,1.0,1.0,1.0,1.0,1.0
219,1.0,1.0,1.0,1.0,1.0
263,1.0,1.0,1.0,1.0,1.0


# From the classification reports we can see that SVM and Decision Tree perform comparatively better than the other classifiers.